In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno
import statsmodels.api as sm
#import tensorflow as tf
#imp
df = pd.read_csv(r"C:\Users\Hamza\Desktop\P2P ortak dosya\Makine Öğrenmesi\Solar1\solar_predict.xls")   #dataframe determined
df.columns=["Date","Time","Power","Load","Temp"]   #the columns in our dataframe
from datetime import datetime
from pytz import timezone
import pytz
df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)  #convert to date
df.drop(['Load'], axis=1, inplace=True)   #I will just look at PV Generation
df['MonthOfYear'] = df.index.strftime('%m').astype(int)
df['DayOfYear'] = df.index.strftime('%j').astype(int)
df['WeekOfYear'] = df.index.strftime('%U').astype(int)
df.index=df.iloc[:,1]
df.index = pd.to_datetime(df.index)
df['TimeOfDay'] = df.index.hour
from pytz import timezone
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import neighbors
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
#from tensorflow.keras.layers import Dense,Activation,Dropout
#from tensorflow.keras.models import Sequential,load_model
#from tensorflow.keras.optimizers import SGD, Adam
#from tensorflow.keras.models import Sequential
from collections import Counter
from scipy import stats
from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge ,Lasso ,ElasticNet, LassoLarsIC
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold,RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale, LabelEncoder  
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from sklearn.svm import SVR
#from catboost import CatBoostRegressor
#from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor
from warnings import filterwarnings
filterwarnings('ignore')
df = pd.read_csv(r"C:\Users\Hamza\Desktop\P2P ortak dosya\Makine Öğrenmesi\Solar1\solar_predict.xls")   #dataframe determined
df.columns=["Date","Time","Power","Load","Temp"]   #the columns in our dataframe
from datetime import datetime
from pytz import timezone
import pytz
df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)  #convert to date
df.drop(['Load'], axis=1, inplace=True)   #I will just look at PV Generation
df['MonthOfYear'] = df.index.strftime('%m').astype(int)
df['DayOfYear'] = df.index.strftime('%j').astype(int)
df['WeekOfYear'] = df.index.strftime('%U').astype(int)
df.index=df.iloc[:,1]
df.index = pd.to_datetime(df.index)
df['TimeOfDay'] = df.index.hour

# SVR

In [8]:
X = df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]]
y = df.iloc[:,2:3]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
lm = sm.OLS(y, X)
model= lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Power   R-squared (uncentered):                   0.506
Model:                            OLS   Adj. R-squared (uncentered):              0.505
Method:                 Least Squares   F-statistic:                              1792.
Date:                Wed, 21 Apr 2021   Prob (F-statistic):                        0.00
Time:                        10:47:44   Log-Likelihood:                         -9466.3
No. Observations:                8760   AIC:                                  1.894e+04
Df Residuals:                    8755   BIC:                                  1.898e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
MonthOfYear    -0.3028      0.023    -13.066      0.000      -0.348      -0.257
DayOfYear       0.0055      0.004      1.422      0.155      -0.002       0.013
WeekOfYear      0.0301      0.027      1.127      0.260      -0.022       0.082
TimeOfDay      -0.0077      0.001     -7.172      0.000      -0.010      -0.006
Temp            0.0538      0.001     56.880      0.000       0.052       0.056
==============================================================================
Omnibus:                      781.176   Durbin-Watson:                   0.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1005.095
Skew:                           0.827   Prob(JB):                    5.58e-219
Kurtosis:                       2.860   Cond. No.                         758.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
sc1 = StandardScaler()
X_s = sc1.fit_transform(X)
sc2 = StandardScaler()
y_s = sc2.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size = 0.3, random_state = 0)
svr_model=SVR("linear").fit(X_train,y_train)
ypred=svr_model.predict(X_test)

In [10]:
svr_params = {"C": [0.1,0.5,1,3]}
svr_cv_model = GridSearchCV(svr_model,svr_params,cv=10).fit(X_train,y_train)
svr_cv_model.best_params_

{'C': 0.5}

In [11]:
svr_cv_model = GridSearchCV(svr_model,svr_params,cv=5,verbose=2).fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................................ C=0.1, total=   0.7s
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ............................................ C=0.1, total=   0.6s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.6s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.6s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.6s
[CV] C=0.5 ...........................................................
[CV] ............................................ C=0.5, total=   1.0s
[CV] C=0.5 ...........................................................
[CV] ............................................ C=0.5, total=   1.0s
[CV] C=0.5 ...........................................................
[CV] ............................................ C=0.5, total=   1.0s
[CV] C=0.5 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   28.4s finished


In [12]:
svr_cv_model.best_params_

{'C': 0.5}

In [16]:
svr_tuned = SVR("poly", C=0.5).fit(X_train,y_train)
ypred=svr_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, ypred)
print('r2 = {}'.format(model))

mse =  0.975624385441695
r2 = 0.0572797928568195


In [20]:
svr_tuned = SVR("rbf", C=0.5).fit(X_train,y_train)
ypred=svr_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, ypred)
print('r2 = {}'.format(model))

mse =  0.4295374476181777
r2 = 0.8172659049176125


In [22]:
svr_tuned = SVR("linear", C=0.5).fit(X_train,y_train)
ypred=svr_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, ypred)
print('r2 = {}'.format(model))

mse =  0.8843027045667986
r2 = 0.22550355417237677


In [24]:
svr_tuned = SVR("sigmoid", C=0.5).fit(X_train,y_train)
ypred=svr_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, ypred)
print('r2 = {}'.format(model))

mse =  60.76286488954037
r2 = -3655.7393629847566
